<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FINAL_BOT_MLLM_MMLM_28P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install ccxt pandas numpy ta-lib matplotlib -q
!pip install ta -q
!pip install anthropic openai -q
!pip install xai-sdk -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.0/130.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.1/641.1 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.6/156.6 kB 4.9 MB/s eta 0:00:00


In [2]:
from google.colab import userdata, drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import ccxt
import pandas as pd
import numpy as np
import time
import smtplib
import ssl
import sqlite3
import os
import math
import pytz
import datetime as dt
import tensorflow as tf
from email.message import EmailMessage
from tensorflow.keras.models import load_model
from ta.volatility import AverageTrueRange as ta_ATR
import json


# Add to imports section at the top
from xai_sdk import Client
from xai_sdk.chat import user, system



# --- LLM INTEGRATION SETUP ---
try:
    from openai import OpenAI
    import anthropic
    from google.colab import userdata

    # 1. API KEY RETRIEVAL (Fetching from Colab Userdata)
    DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    # Use the key variable name requested by the user for Claude
    CLAUDE_API_KEY = userdata.get('ANTHROPIC_API_KEY')

    XAI_KEY = userdata.get('XAI_KEY')  # NEW: Grok API key

    # 2. CLIENT AND MODEL INITIALIZATION

    # DeepSeek Client (Uses OpenAI library structure with base_url)
    LLM_CLIENT_DEEPSEEK = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")


    LLM_MODEL_GPT = "gpt-4o" # Reliable and fast
    api_key = userdata.get('OPENAI_API_KEY')
    LLM_CLIENT_GPT = OpenAI(api_key=api_key)

    # Claude Client (Dedicated Anthropic library setup for Claude Opus 4)
    LLM_CLIENT_CLAUDE = anthropic.Anthropic(api_key=CLAUDE_API_KEY)
    #LLM_MODEL_CLAUDE = "claude-opus-4-20250514" # User's preferred model
    LLM_MODEL_CLAUDE =  "claude-opus-4-20250514"

    print("✅ All three LLM clients initialized (DeepSeek, Claude, GPT)")

except Exception as e:
    # Define Dummy Client to prevent crashes if API keys or imports fail
    class DummyLLMClient:
        def chat(self): pass
        def chat_completions(self): pass
        def messages(self):
            class DummyMessages:
                def create(self, **kwargs):
                    # Simulate an API failure
                    raise Exception("LLM Client not initialized - DUMMY API failure.")
            return DummyMessages()

    LLM_CLIENT_DEEPSEEK = DummyLLMClient()
    LLM_CLIENT_GPT = DummyLLMClient()
    LLM_CLIENT_CLAUDE = DummyLLMClient()
    LLM_MODEL_GPT = "dummy"
    LLM_MODEL_CLAUDE = "dummy"

    print(f"WARNING: LLM client failed to initialize: {e}. Using simulated/fallback scores.")


# --- GLOBAL PREDICTION FUNCTION (from original code) ---
@tf.function(reduce_retracing=True)
def global_predict(model, input_data):
    """Global prediction function to prevent TensorFlow retracing warnings"""
    try:
        input_data = tf.convert_to_tensor(input_data, dtype=tf.float32)
        return model(input_data, training=False)
    except Exception as e:
        print(f"❌ Global predict error: {e}")
        raise


# --- CONFIGURATION & CONSTANTS (from original code) ---
LIVE_MODE = False
VIRTUAL_BUY_CAPITAL = 500.00
VIRTUAL_SELL_CAPITAL = 500.00
TOTAL_START_CAPITAL = VIRTUAL_BUY_CAPITAL + VIRTUAL_SELL_CAPITAL
TIMEFRAME = '1h'
LOOKBACK_WINDOW_SIZE = 720 # Global Default (Overridden by WFO_LOOK_BACK)
TIMEZONE = pytz.timezone('America/New_York')
HOURS_PER_DAY = 24
DAYS_PER_YEAR = 365
SHARPE_ANNUALIZATION_FACTOR = np.sqrt(HOURS_PER_DAY * DAYS_PER_YEAR)
MIN_LIQUID_CAPITAL_FOR_TRADE = 100.0
MAX_CONSECUTIVE_ERRORS = 5
MAX_ASSET_CORRELATION_COUNT = 3
TRADE_SLEEP_SECONDS = 1800  # 30 minutes
GLOBAL_MAX_DRAWDOWN_PCT = 0.3  # Aligned with WFO max drawdowns
DB_PATH = '/content/drive/MyDrive/TradingBotLogs/trading_bot_MLLM.db'
# drive.mount('/content/drive') # Already mounted above
VIRTUAL_ALLOCATED_BUY = 0.0
VIRTUAL_ALLOCATED_SELL = 0.0
CYCLE_PNL_BUY = 0.0
CYCLE_PNL_SELL = 0.0
POSITION_INFO = {}
LOADED_MODELS = {}
TRADE_HISTORY = []
error_counters = {}
disabled_assets = set()
last_portfolio_value = TOTAL_START_CAPITAL
PERFORMANCE_METRICS = {
    'total_trades': 0,
    'winning_trades': 0,
    'total_pnl': 0.0,
    'asset_performance': {},
    'hourly_returns': [],
    'max_portfolio_value': TOTAL_START_CAPITAL,
    'max_drawdown': 0.0,
    'start_time': time.time()
}
VIRTUAL_BUY_START = VIRTUAL_BUY_CAPITAL
VIRTUAL_SELL_START = VIRTUAL_SELL_CAPITAL
VIRTUAL_BUY_PEAK = VIRTUAL_BUY_START
VIRTUAL_SELL_PEAK = VIRTUAL_SELL_START
MDD_PROTECT_BUY = False
MDD_PROTECT_SELL = False

try:
    KRAKEN_API_KEY = userdata.get('KRAKENDANKA')
    KRAKEN_SECRET = userdata.get('KRAKENDANKASECRET')
    EMAIL_PASSWORD = userdata.get('EMAIL_PASSWORD')
    SENDER_EMAIL = userdata.get('EMAIL_SENDER')
    RECIPIENT_EMAIL = userdata.get('EMAIL_RECIPIENT')
    SMTP_SERVER = userdata.get('EMAIL_SMTP_SERVER')
    SMTP_PORT = int(userdata.get('EMAIL_SMTP_PORT'))
except:
    KRAKEN_API_KEY = 'DUMMY_KEY'
    KRAKEN_SECRET = 'DUMMY_SECRET'
    EMAIL_PASSWORD = 'DUMMY_PASS'
    SENDER_EMAIL = 'dummy@example.com'
    RECIPIENT_EMAIL = 'recipient@example.com'
    SMTP_SERVER = 'smtp.example.com'
    SMTP_PORT = 587

##BTC - CNN-LSTM with dataset TRAIN:12.5Y
#### WFO RESULTS - 2.3 years historical-data 2023-2025 WITH TUNER-HYPERBAND

#--- Aggregate Performance ---
#Average Out-of-Sample Sharpe Ratio: 6.08
#Total Compounded Return: 1842.32%
#Worst Out-of-Sample Max Drawdown: 30.14%
#Total Trades: 510


## ETH - CNN-LSTM with dataset TRAIN:9.5Y
#### WFO RESULTS - 2.3 years historical-data 2023-2025 WITH TUNER-HYPERBAND

#--- Aggregate Performance ---
#Average Out-of-Sample Sharpe Ratio: 6.20
#Total Compounded Return: 1757.69%
#Worst Out-of-Sample Max Drawdown: 30.39%
#Total Trades: 432


##SOL -  CNN-LSTM with dataset TRAIN:3.5Y
#### WFO RESULTS - 2.3 years historical-data 2023-2025 WITH TUNER-HYPERBAND

#--- Aggregate Performance ---
#Average Out-of-Sample Sharpe Ratio: 5.10
#Total Compounded Return: 1185.53%
#Worst Out-of-Sample Max Drawdown: 32.68%
#Total Trades: 430

# --- ASSET PROFILES (from original code) ---
ASSET_PROFILES = {
    "BTC/USD": {
        "model_path": '/content/drive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_MLM12_BTC.keras',
        "volatility_filter_low": 0.1, "volatility_filter_high": 1500.0,
        "P": {
            'CONFIDENCE_THRESHOLD': 0.015,
            'LLM_VETO_THRESHOLD': 0.4,
            'POOR_CLASSIFIER_THRESHOLD': 0.5,
            'ATR_TP': 3.0,
            'ATR_SL': 0.70,
            'MAX_POS_SIZE': 0.15,
            'BREAKEVEN_ATR': 0.5,
            'TRAILING_STOP_MULT': 0.06,
            'RISK_PER_TRADE': 0.006,
            'MAX_HOLD_PERIODS': 96,
            'MIN_ATR_THRESHOLD': 0.05,
            'WFO_MIN_RISK_REWARD': 0.1,
            'WFO_DYNAMIC_POS_SIZING': 'hybrid',
            'WFO_VOLATILITY_FACTOR': 0.02,
            'WFO_PROFIT_LOCK_ATR': 0.5,
            'WFO_LOOK_BACK': 72
        }
    },


    "ETH/USD": {
        "model_path": '/content/drive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_MLM12_ETH.keras',
        "volatility_filter_low": 0.1, "volatility_filter_high": 50.0,
        "P": {
            'CONFIDENCE_THRESHOLD': 0.01,
            'LLM_VETO_THRESHOLD': 0.30000000000000004,
            'POOR_CLASSIFIER_THRESHOLD': 0.5,
            'ATR_TP': 3.5,
            'ATR_SL': 0.4,
            'MAX_POS_SIZE': 0.1,
            'BREAKEVEN_ATR': 0.4,
            'TRAILING_STOP_MULT': 0.07,
            'RISK_PER_TRADE': 0.008,
            'MAX_HOLD_PERIODS': 120,
            'MIN_ATR_THRESHOLD': 0.05,
            'WFO_PROFIT_LOCK_ATR': 0.4,
            'WFO_VOLATILITY_FACTOR': 0.01,
            'WFO_LOOK_BACK': 72,
            'WFO_DYNAMIC_POS_SIZING': 'risk_based',
            'WFO_MIN_RISK_REWARD': 0.2
        }
    },

    "SOL/USD": {
        "model_path": '/content/drive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_MLM12_SOL.keras',
        "volatility_filter_low": 0.01, "volatility_filter_high": 5.0,
        "P": {
            'CONFIDENCE_THRESHOLD': 0.005,
            'LLM_VETO_THRESHOLD': 0.30000000000000004,
            'POOR_CLASSIFIER_THRESHOLD': 0.6,
            'ATR_TP': 3.5,
            'ATR_SL': 0.6,
            'MAX_POS_SIZE': 0.1,
            'BREAKEVEN_ATR': 0.4,
            'TRAILING_STOP_MULT': 0.07,
            'RISK_PER_TRADE': 0.005,
            'MAX_HOLD_PERIODS': 120,
            'MIN_ATR_THRESHOLD': 0.1,
            'WFO_PROFIT_LOCK_ATR': 0.4,
            'WFO_VOLATILITY_FACTOR': 0.015,
            'WFO_LOOK_BACK': 72,
            'WFO_DYNAMIC_POS_SIZING': 'risk_based',
            'WFO_MIN_RISK_REWARD': 0.2
        }
    }
}

for symbol in ASSET_PROFILES.keys():
    PERFORMANCE_METRICS['asset_performance'][symbol] = {'trades': 0, 'pnl': 0.0}

# --- DATABASE FUNCTIONS (from original code) ---
def init_database():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS candles (
            id INTEGER PRIMARY KEY AUTOINCREMENT, symbol TEXT NOT NULL,
            timestamp INTEGER NOT NULL, open REAL NOT NULL, high REAL NOT NULL,
            low REAL NOT NULL, close REAL NOT NULL, volume REAL NOT NULL,
            atr REAL, created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(symbol, timestamp)
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS technical_indicators (
            id INTEGER PRIMARY KEY AUTOINCREMENT, symbol TEXT NOT NULL,
            timestamp INTEGER NOT NULL, rsi_14 REAL, macd REAL, macd_signal REAL,
            bb_upper REAL, bb_middle REAL, bb_lower REAL, volume_sma_20 REAL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, UNIQUE(symbol, timestamp)
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS trade_history (\
            id INTEGER PRIMARY KEY AUTOINCREMENT, symbol TEXT NOT NULL,\
            side TEXT NOT NULL, entry_price REAL, exit_price REAL,\
            size REAL, pnl REAL, exit_reason TEXT, duration_hours REAL,\
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP\
        )\
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS cycle_performance (\
            id INTEGER PRIMARY KEY AUTOINCREMENT, cycle_number INTEGER NOT NULL,\
            timestamp TEXT NOT NULL, portfolio_value REAL, total_return_pct REAL,\
            max_drawdown_pct REAL, sharpe_ratio REAL, total_trades INTEGER,\
            winning_trades INTEGER, total_pnl REAL, runtime_hours REAL,\
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, UNIQUE(cycle_number)\
        )\
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS asset_cycle_performance (\
            id INTEGER PRIMARY KEY AUTOINCREMENT, cycle_number INTEGER NOT NULL,\
            symbol TEXT NOT NULL, trades INTEGER, pnl REAL,\
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, UNIQUE(cycle_number, symbol)\
        )\
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS bot_state (\
            id INTEGER PRIMARY KEY CHECK (id = 1),\
            cycle_number INTEGER NOT NULL,\
            virtual_buy_capital REAL NOT NULL,\
            virtual_sell_capital REAL NOT NULL,\
            position_info TEXT,\
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP\
        )\
    ''')
    conn.commit()
    conn.close()
    print("✅ Database initialized with performance and state tables")

init_database()

def save_candles_to_db(symbol, df):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    new_candles = updated_candles = 0
    for index, row in df.iterrows():
        try:
            cursor.execute('''
                INSERT OR REPLACE INTO candles
                (symbol, timestamp, open, high, low, close, volume, atr)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (symbol, int(row['timestamp']), row['open'], row['high'],
                  row['low'], row['close'], row['volume'], row.get('ATR', 0)))
            if cursor.rowcount == 1: new_candles += 1
            else: updated_candles += 1
        except: continue
    conn.commit()
    conn.close()
    print(f"💾 {symbol}: {new_candles} new, {updated_candles} updated candles")

def save_bot_state(cycle_count):
    global VIRTUAL_BUY_CAPITAL, VIRTUAL_SELL_CAPITAL, POSITION_INFO
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    position_info_json = json.dumps(POSITION_INFO)
    cursor.execute('''
        INSERT OR REPLACE INTO bot_state
        (id, cycle_number, virtual_buy_capital, virtual_sell_capital, position_info)
        VALUES (1, ?, ?, ?, ?)
    ''', (int(cycle_count), VIRTUAL_BUY_CAPITAL, VIRTUAL_SELL_CAPITAL, position_info_json))
    conn.commit()
    conn.close()

def save_cycle_performance(cycle_count, current_time):
    global PERFORMANCE_METRICS
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    portfolio_value = VIRTUAL_BUY_CAPITAL + VIRTUAL_SELL_CAPITAL
    total_return = (portfolio_value - TOTAL_START_CAPITAL) / TOTAL_START_CAPITAL * 100
    max_drawdown = PERFORMANCE_METRICS['max_drawdown'] * 100
    sharpe_raw = (np.mean(PERFORMANCE_METRICS['hourly_returns']) / np.std(PERFORMANCE_METRICS['hourly_returns']) * SHARPE_ANNUALIZATION_FACTOR
                  if PERFORMANCE_METRICS['hourly_returns'] and np.std(PERFORMANCE_METRICS['hourly_returns']) > 0 else 0)
    runtime_hours = (time.time() - PERFORMANCE_METRICS['start_time']) / 3600
    cursor.execute('''
        INSERT OR REPLACE INTO cycle_performance
        (cycle_number, timestamp, portfolio_value, total_return_pct, max_drawdown_pct,
         sharpe_ratio, total_trades, winning_trades, total_pnl, runtime_hours)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (cycle_count, get_timestamp(current_time), portfolio_value, total_return,
          max_drawdown, sharpe_raw, PERFORMANCE_METRICS['total_trades'],
          PERFORMANCE_METRICS['winning_trades'], PERFORMANCE_METRICS['total_pnl'],
          runtime_hours))
    for symbol in ASSET_PROFILES.keys():
        trades = PERFORMANCE_METRICS['asset_performance'][symbol]['trades']
        pnl = PERFORMANCE_METRICS['asset_performance'][symbol]['pnl']
        cursor.execute('''
            INSERT OR REPLACE INTO asset_cycle_performance
            (cycle_number, symbol, trades, pnl)
            VALUES (?, ?, ?, ?)
        ''', (cycle_count, symbol, trades, pnl))
    conn.commit()
    conn.close()
    save_bot_state(cycle_count)
    print(f"💾 Cycle {cycle_count} performance saved to database")

def load_bot_state():
    global VIRTUAL_BUY_CAPITAL, VIRTUAL_SELL_CAPITAL, POSITION_INFO, VIRTUAL_ALLOCATED_BUY, VIRTUAL_ALLOCATED_SELL
    global VIRTUAL_BUY_PEAK, VIRTUAL_SELL_PEAK, last_portfolio_value
    conn = sqlite3.connect(DB_PATH)
    query = 'SELECT cycle_number, virtual_buy_capital, virtual_sell_capital, position_info FROM bot_state WHERE id = 1'
    df = pd.read_sql_query(query, conn)
    conn.close()
    if df.empty:
        print("🔄 No previous state found. Starting from Cycle 0.")
        return 0
    state = df.iloc[0]
    VIRTUAL_BUY_CAPITAL = state['virtual_buy_capital']
    VIRTUAL_SELL_CAPITAL = state['virtual_sell_capital']
    try:
        POSITION_INFO = json.loads(state['position_info'])
        VIRTUAL_ALLOCATED_BUY = sum(t['allocated_amount'] for t in POSITION_INFO.values() if t['side'] == 'buy')
        VIRTUAL_ALLOCATED_SELL = sum(t['allocated_amount'] for t in POSITION_INFO.values() if t['side'] == 'sell')
        VIRTUAL_BUY_PEAK = VIRTUAL_BUY_CAPITAL
        VIRTUAL_SELL_PEAK = VIRTUAL_SELL_CAPITAL
        last_portfolio_value = VIRTUAL_BUY_CAPITAL + VIRTUAL_SELL_CAPITAL
    except json.JSONDecodeError as e:
        print(f"❌ State loading error (JSON Decode): {e}. Starting fresh.")
        return 0
    except Exception as e:
        print(f"❌ State loading error (General): {e}. Starting fresh.")
        return 0
    cycle_number = state['cycle_number']
    if isinstance(cycle_number, bytes):
        try:
            cycle_number = int.from_bytes(cycle_number, byteorder='little')
        except ValueError as e:
            print(f"❌ Cycle number decoding error: {e}. Starting fresh.")
            return 0
    else:
        try:
            cycle_number = int(cycle_number)
        except ValueError as e:
            print(f"❌ Cycle number conversion error: {e}. Starting fresh.")
            return 0
    print(f"✅ State loaded. Resuming from Cycle {cycle_number + 1}.")
    print(f"  > Capital: BUY ${VIRTUAL_BUY_CAPITAL:,.2f} / SELL ${VIRTUAL_SELL_CAPITAL:,.2f}")
    print(f"  > Open Positions: {len(POSITION_INFO)}")
    return cycle_number

def get_historical_data(symbol, lookback_candles=LOOKBACK_WINDOW_SIZE):
    conn = sqlite3.connect(DB_PATH)
    query = '''SELECT timestamp, open, high, low, close, volume, atr FROM candles
               WHERE symbol = ? ORDER BY timestamp DESC LIMIT ?'''
    df = pd.read_sql_query(query, conn, params=[symbol, lookback_candles])
    conn.close()
    if not df.empty:
        df = df.iloc[::-1]
        df.reset_index(drop=True, inplace=True)
    return df

# --- CORE FUNCTIONS (from original code) ---
def get_timestamp(dt_obj=None):
    if dt_obj is None: dt_obj = dt.datetime.now(TIMEZONE)
    return dt_obj.isoformat()

def send_email_alert(subject, body):
    mode_tag = "[LIVE]" if LIVE_MODE else "[DRY RUN]"
    subject = f"{mode_tag} {subject}"
    try:
        msg = EmailMessage()
        msg.set_content(body)
        msg['Subject'] = subject
        msg['From'] = SENDER_EMAIL
        msg['To'] = RECIPIENT_EMAIL
        context = ssl.create_default_context()
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls(context=context)
            server.login(SENDER_EMAIL, EMAIL_PASSWORD)
            server.send_message(msg)
        print(f"📧 Alert: {subject}")
    except Exception as e:
        print(f"Warning: Failed to send email alert: {e}")

def get_liquid_usd_equity(exchange, side, symbol=None):
    if not LIVE_MODE:
        if side == 'buy': return VIRTUAL_BUY_CAPITAL - VIRTUAL_ALLOCATED_BUY
        elif side == 'sell': return VIRTUAL_SELL_CAPITAL - VIRTUAL_ALLOCATED_SELL
        return 0.0
    try:
        balance = exchange.fetch_balance()
        if side == 'buy':
            return balance['free'].get('USD', 0.0) or balance['free'].get('ZUSD', 0.0)
        elif side == 'sell' and symbol:
            base_asset = symbol.split('/')[0]
            return balance['free'].get(base_asset, 0.0)
    except:
        return 0.0
    return 0.0

def initialize_exchange():
    global LOADED_MODELS, error_counters
    exchange = ccxt.kraken({'apiKey': KRAKEN_API_KEY, 'secret': KRAKEN_SECRET, 'enableRateLimit': True})
    error_counters = {symbol: 0 for symbol in ASSET_PROFILES}
    for symbol, profile in ASSET_PROFILES.items():
        try:
            model_path = profile['model_path']
            if os.path.exists(model_path):
                LOADED_MODELS[symbol] = load_model(model_path)
                print(f"✅ Model loaded for {symbol}")
            else:
                print(f"❌ Model file not found: {model_path}")
                LOADED_MODELS[symbol] = None
        except Exception as e:
            print(f"❌ Failed to load model for {symbol}: {e}")
            LOADED_MODELS[symbol] = None
    if not LIVE_MODE:
        print(f"{get_timestamp()} INFO: DRY RUN MODE. TOTAL VIRTUAL CAPITAL: ${TOTAL_START_CAPITAL:,.2f}")
    exchange.load_markets()
    return exchange

def fetch_data_with_history(exchange, symbol):
    try:
        # Fetch up to the max global lookback just to ensure data buffer is large enough
        historical_df = get_historical_data(symbol, LOOKBACK_WINDOW_SIZE)
        ohlcv = exchange.fetch_ohlcv(symbol, TIMEFRAME, limit=LOOKBACK_WINDOW_SIZE)
        latest_df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        latest_df['ATR'] = ta_ATR(latest_df['high'], latest_df['low'], latest_df['close'], window=14).average_true_range()
        save_candles_to_db(symbol, latest_df)
        print(f"✅ {symbol}: {len(latest_df)} candles processed")
        return latest_df
    except Exception as e:
        print(f"❌ Data fetch error for {symbol}: {e}")
        historical_df = get_historical_data(symbol, LOOKBACK_WINDOW_SIZE)
        if not historical_df.empty:
            print(f"🔄 Using historical data from DB for {symbol}")
            return historical_df
        return None

def prepare_model_features(df, symbol):
    # Use asset-specific lookback from WFO parameters
    P = ASSET_PROFILES[symbol]['P']
    look_back_wfo = P.get('WFO_LOOK_BACK', LOOKBACK_WINDOW_SIZE)

    if df is None or len(df) < look_back_wfo:
        print(f"❌ Feature preparation failed: Not enough data for WFO lookback {look_back_wfo}")
        return None

    # Slice the DataFrame to the WFO lookback period
    df_sliced = df.iloc[-look_back_wfo:].copy()

    try:
        features = []
        features.append(df_sliced['open'].values)
        features.append(df_sliced['high'].values)
        features.append(df_sliced['low'].values)
        features.append(df_sliced['close'].values)
        features.append(df_sliced['volume'].values)
        features.append(df_sliced['ATR'].values)

        # Calculate Technical Indicators on the sliced data
        delta = df_sliced['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        features.append(rsi.fillna(50).values)
        features.append(df_sliced['close'].rolling(window=20).mean().fillna(df_sliced['close']).values)
        features.append(df_sliced['close'].rolling(window=50).mean().fillna(df_sliced['close']).values)
        high_20 = df_sliced['high'].rolling(window=20).max()
        low_20 = df_sliced['low'].rolling(window=20).min()
        price_position = (df_sliced['close'] - low_20) / (high_20 - low_20)
        features.append(price_position.fillna(0.5).values)
        volume_ma = df_sliced['volume'].rolling(window=20).mean()
        volume_ratio = df_sliced['volume'] / volume_ma
        features.append(volume_ratio.fillna(1).values)
        features.append(df_sliced['close'].pct_change().rolling(window=20).std().fillna(0).values)

        feature_matrix = np.column_stack(features).astype(np.float32)
        feature_matrix = (feature_matrix - feature_matrix.mean(axis=0)) / (feature_matrix.std(axis=0) + 1e-8)
        feature_matrix = feature_matrix.reshape(1, feature_matrix.shape[0], feature_matrix.shape[1])
        print(f"✅ Feature matrix shape: {feature_matrix.shape} (WFO Lookback: {look_back_wfo})")
        return feature_matrix
    except Exception as e:
        print(f"❌ Feature preparation error: {e}")
        return None

def decode_prediction(prediction):
    try:
        if hasattr(prediction, 'numpy'):
            pred_array = prediction.numpy()[0]
        else:
            pred_array = prediction[0] if len(prediction.shape) > 1 else prediction
        if len(pred_array) == 3:
            exp_values = np.exp(pred_array - np.max(pred_array))
            probabilities = exp_values / np.sum(exp_values)
            buy_prob = float(probabilities[0])
            sell_prob = float(probabilities[1])
            hold_prob = float(probabilities[2])
            soft_cap = 0.9999
            buy_prob = min(buy_prob, soft_cap)
            sell_prob = min(sell_prob, soft_cap)
            hold_prob = min(hold_prob, soft_cap)
            return buy_prob, sell_prob, hold_prob
        elif len(pred_array) == 1:
            signal_value = pred_array[0]
            conviction = abs(signal_value)
            if conviction >= 1.0: conviction = 0.9999
            if signal_value > 0:
                return conviction, 0.0, 1.0 - conviction
            elif signal_value < 0:
                return 0.0, conviction, 1.0 - conviction
            else:
                return 0.0, 0.0, 1.0
    except Exception as e:
        print(f"❌ Decoding error: {e}")
        return 0.0, 0.0, 1.0

def predict_signal(df, symbol, P):
    if symbol not in LOADED_MODELS or LOADED_MODELS[symbol] is None:
        return 'HOLD', 0.0, [0.0, 0.0, 1.0]
    try:
        features = prepare_model_features(df, symbol) # Passed symbol for WFO Lookback
        if features is None:
            return 'HOLD', 0.0, [0.0, 0.0, 1.0]
        expected_features = LOADED_MODELS[symbol].input_shape[-1]
        actual_features = features.shape[-1]
        if actual_features != expected_features:
            if actual_features < expected_features:
                padding = np.zeros((features.shape[0], features.shape[1], expected_features - actual_features), dtype=np.float32)
                features = np.concatenate([features, padding], axis=-1)
            else:
                features = features[:, :, :expected_features]
        prediction = global_predict(LOADED_MODELS[symbol], features)
        buy_prob, sell_prob, hold_prob = decode_prediction(prediction)
        print(f"📈 {symbol} Prediction Probabilities: BUY={buy_prob:.3f}, SELL={sell_prob:.3f}, HOLD={hold_prob:.3f}")
        poor_classifier_threshold = P.get('POOR_CLASSIFIER_THRESHOLD', 0.5)
        if hold_prob >= poor_classifier_threshold:
            print(f"🎯 POOR CLASSIFIER: {symbol} - HOLD probability {hold_prob:.3f} >= {poor_classifier_threshold}")
            return 'HOLD', hold_prob, [buy_prob, sell_prob, hold_prob]
        if buy_prob > sell_prob:
            signal = 'BUY'
            confidence = buy_prob
        else:
            signal = 'SELL'
            confidence = sell_prob
        if confidence > P['CONFIDENCE_THRESHOLD']:
            print(f"🤖 {symbol} ML Prediction: {signal} (Confidence: {confidence:.3f})\n")
            return signal, confidence, [buy_prob, sell_prob, hold_prob]
        else:
            return 'HOLD', confidence, [buy_prob, sell_prob, hold_prob]
    except Exception as e:
        print(f"❌ Prediction error for {symbol}: {e}")
        return 'HOLD', 0.0, [0.0, 0.0, 1.0]

def get_comprehensive_technical_analysis(symbol, df):
    if df is None or len(df) < LOOKBACK_WINDOW_SIZE: return f"Building analysis for {symbol}"
    try:
        current_price = df['close'].iloc[-1]
        price_24h_change = ((current_price - df['close'].iloc[-HOURS_PER_DAY]) / df['close'].iloc[-HOURS_PER_DAY]) * 100
        volume_ratio = df['volume'].iloc[-1] / df['volume'].tail(HOURS_PER_DAY).mean()
        atr_pct = (df['ATR'].iloc[-1] / current_price) * 100
        ma_20 = df['close'].tail(20).mean()
        ma_50 = df['close'].tail(50).mean()
        ma_status = 'above' if current_price > ma_20 else 'below'
        ma_trend = 'Bullish' if ma_20 > ma_50 else 'Bearish'
        analysis_parts = [
            f"Price: ${current_price:.2f} ({price_24h_change:+.2f}% 24h)",
            f"Volume: {volume_ratio:.1f}x avg",
            f"ATR: {atr_pct:.3f}% of price",
            f"Position: {ma_status} 20MA (20/50MA Trend: {ma_trend})"
        ]
        return " | ".join(analysis_parts)
    except: return f"Technical analysis for {symbol}"

def calculate_technical_sentiment(df):
    if df is None or len(df) < 20: return 0.0
    try:
        current_price = df['close'].iloc[-1]
        price_24h_change = (df['close'].iloc[-1] - df['close'].iloc[-HOURS_PER_DAY]) / df['close'].iloc[-HOURS_PER_DAY] if len(df) >= HOURS_PER_DAY else 0
        current_volume = df['volume'].iloc[-1]
        avg_volume = df['volume'].tail(72).mean()
        volume_ratio = min(current_volume / avg_volume, 3)
        factors = [
            price_24h_change * 0.4,
            (volume_ratio - 1) * 0.2,
            -abs(price_24h_change) / ((df['ATR'].iloc[-1] / current_price * 100) + 0.1) * 0.2,
            (1 if (current_price > df['close'].tail(20).mean() > df['close'].tail(50).mean()) else -1 if (current_price < df['close'].tail(20).mean() < df['close'].tail(50).mean()) else 0) * 0.2
        ]
        raw_sentiment = sum(factors)
        return max(-1.0, min(1.0, raw_sentiment * 5))
    except: return 0.0

# Helper function to query a single LLM and parse the score
def query_single_llm(client_type, client, model_name, system_prompt, market_analysis):
    """Queries a single LLM and attempts to return a float score between -1.0 and 1.0."""
    try:
        # Anthropic Client Call
        if client_type == 'anthropic':
            response = client.messages.create(
                model=model_name,
                max_tokens=20, # Keep output short
                temperature=0.1, # Keep low for consistent sentiment
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": market_analysis}
                ]
            )
            # The structure for Anthropic response.content is a list of blocks
            score_text = response.content[0].text.strip()

        # OpenAI Compatible Client Call (GPT and DeepSeek)
        else:
            response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": market_analysis}
                ],
                temperature=0.1
            )
            score_text = response.choices[0].message.content.strip()

        # Score parsing logic (robust to sign and clamping)
        score_text_clean = score_text.replace('+', '').replace('-', '').strip()
        try:
            score = float(score_text_clean)
        except ValueError:
            print(f"❌ {model_name} returned non-numeric score: '{score_text}'.")
            return None

        # Clamp the score to [-1.0, 1.0]
        score = max(-1.0, min(1.0, score))

        # Apply the original sign if negative was present in the raw text
        if '-' in score_text:
            score *= -1

        return score

    except Exception as e:
        # Fallback to general exception handling
        print(f"❌ {model_name} query failed: {e}")
        return None



import re # Make sure 'import re' is included at the top of your script or within the function

def get_llm_sentiment_score(symbol: str, df: pd.DataFrame = None) -> float:
    global LLM_MODEL_GPT, LLM_MODEL_CLAUDE

    # --- Check for Model Feature Readiness ---
    if df is None or len(df) < 50:
        print(f"❌ LLM analysis skipped: Not enough data for {symbol}")
        return calculate_technical_sentiment(df) # Fallback

    market_analysis = get_comprehensive_technical_analysis(symbol, df)
    system_prompt = "You are an expert crypto technical analyst. Analyze the technical data and return ONLY a decimal score between -1.0 (Extremely Bearish) and +1.0 (Extremely Bullish). Do not include any text, markdown, or justification. Focus on price momentum, volume, volatility, and market structure."

    llm_results = []

    print(f"🧠 LLM Prompt Summary: {market_analysis}")

    # Helper function for robust score extraction
    def extract_score(score_text):
        # Regex to find a floating-point number (optional sign, with or without leading zero/decimal)
        match = re.search(r'^-?\d*\.?\d+', score_text.strip())
        if match:
            score = float(match.group(0))
            return max(-1.0, min(1.0, score))
        return None

    # 1. Query DeepSeek-Reasoner (OpenAI-compatible)
    score_ds = None
    try:
        response = LLM_CLIENT_DEEPSEEK.chat.completions.create(
            model="deepseek-reasoner",
            messages=[{"role": "system", "content": system_prompt},
                     {"role": "user", "content": market_analysis}],
            temperature=0.1
        )
        score_text = response.choices[0].message.content.strip()
        score_ds = extract_score(score_text)

        if score_ds is not None:
            llm_results.append(score_ds)
            print(f"🧠 DeepSeek Score: {score_ds:.2f}")
        else:
            print(f"❌ DeepSeek returned non-numeric score: '{score_text}'.")
    except Exception as e:
        print(f"⚠️ DeepSeek query failed: {e}")

    # 2. Query Claude Opus 4 (Anthropic-specific)
    score_cl = None
    try:
        response = LLM_CLIENT_CLAUDE.messages.create(
            model=LLM_MODEL_CLAUDE,
            max_tokens=30,
            temperature=0.1,
            system=system_prompt,
            messages=[
                {"role": "user", "content": market_analysis}
            ]
        )

        score_text = response.content[0].text.strip()
        score_cl = extract_score(score_text)

        if score_cl is not None:
            llm_results.append(score_cl)
            print(f"🧠 Claude Score: {score_cl:.2f}")
        else:
            print(f"❌ Claude returned non-numeric score: '{score_text}'.")

    except Exception as e:
        print(f"⚠️ Claude query failed: {e}")

    # 3. Query GPT-4o (JSON format)
    score_gpt = None
    gpt_system_prompt_json = "You are an expert crypto technical analyst. Analyze the technical data. You MUST return a JSON object with a single key 'score'. The value of 'score' MUST be a decimal number between -1.0 (Extremely Bearish) and +1.0 (Extremely Bullish). For example: {'score': 0.75}."

    gpt_messages_json = [
        {"role": "system", "content": gpt_system_prompt_json},
        {"role": "user", "content": market_analysis}
    ]

    try:
        response = LLM_CLIENT_GPT.chat.completions.create(
            model=LLM_MODEL_GPT,
            messages=gpt_messages_json,
            max_completion_tokens=50,
            response_format={"type": "json_object"}
        )

        score_text = response.choices[0].message.content.strip()
        data = json.loads(score_text)
        raw_score = data.get('score')

        if raw_score is not None:
            score_gpt = float(raw_score)
            score_gpt = max(-1.0, min(1.0, score_gpt))
            llm_results.append(score_gpt)
            print(f"🧠 GPT Score: {score_gpt:.2f}")
        else:
            print(f"❌ GPT returned malformed JSON or missing score key: '{score_text}'.")

    except json.JSONDecodeError:
        print(f"❌ GPT returned non-JSON data: '{score_text}'.")
    except Exception as e:
        print(f"⚠️ GPT query failed: {e}")

    # 4. Query Grok-4 (NEW ADDITION)
    score_grok = None
    try:
        # Initialize Grok client
        from xai_sdk import Client
        from xai_sdk.chat import user, system
        from google.colab import userdata

        XAI_key = userdata.get('XAI_KEY')
        grok_client = Client(api_host="api.x.ai", api_key=XAI_key)

        # Create chat session
        chat = grok_client.chat.create(model="grok-4-0709", temperature=0.1)

        # Append messages
        chat.append(system(system_prompt))
        chat.append(user(market_analysis))

        # Get response
        response = chat.sample()
        score_text = response.content.strip()
        score_grok = extract_score(score_text)

        if score_grok is not None:
            score_grok = float(score_grok)
            score_grok = max(-1.0, min(1.0, score_grok))  # Clamp to [-1, 1]

            llm_results.append(score_grok)
            print(f"🧠 Grok Score: {score_grok:.2f}")
        else:
            print(f"❌ Grok returned non-numeric score: '{score_text}'.")

    except Exception as e:
        print(f"⚠️ Grok query failed: {e}")

    # --- Final Ensemble ---
    print("\n")
    if len(llm_results) < 2:
        print("❌ Less than 2 LLM queries were successful. Falling back to technical sentiment.")
        return calculate_technical_sentiment(df)

    # Final Ensemble Score: The Average (now from 4 models)
    ensemble_score = np.mean(llm_results)
    print(f"🧠 Ensemble LLM Score for {symbol}: {ensemble_score:.2f} (from {len(llm_results)} models)")
    return ensemble_score

def apply_volatility_filter(df, symbol):
    config = ASSET_PROFILES[symbol]
    returns = df['close'].pct_change().dropna()
    recent_volatility = returns.tail(HOURS_PER_DAY).std() * 100
    print(f"🔍 {symbol} Volatility: {recent_volatility:.2f}% (Low: {config['volatility_filter_low']:.2f}%, High: {config['volatility_filter_high']:.2f}%)\n")
    if recent_volatility < config['volatility_filter_low']:
        print(f"🚫 {symbol}: Volatility too low ({recent_volatility:.2f}%)")
        return False
    elif recent_volatility > config['volatility_filter_high']:
        print(f"🚫 {symbol}: Volatility too high ({recent_volatility:.2f}%)")
        return False
    return True

def calculate_portfolio_correlation():
    active_positions = list(POSITION_INFO.keys())
    if len(active_positions) > MAX_ASSET_CORRELATION_COUNT:
        print(f"⚠️ CORRELATION ALERT: {len(active_positions)} assets active")
        return False
    return True

def calculate_position_size(exchange, current_price, atr_value, P, config, side, symbol):
    liquid_equity = get_liquid_usd_equity(exchange, side)

    # 1. Minimum Capital and ATR Check
    if liquid_equity < MIN_LIQUID_CAPITAL_FOR_TRADE:
        print(f"🚫 POS SIZING BLOCKED: {side.upper()} {symbol}. Liquid Equity (${liquid_equity:,.2f}) < Minimum (${MIN_LIQUID_CAPITAL_FOR_TRADE:,.2f}).")
        return 0, 0, liquid_equity
    if atr_value < P['MIN_ATR_THRESHOLD']:
        print(f"🚫 POS SIZING BLOCKED: {side.upper()} {symbol}. ATR ({atr_value:.3f}) < Min Threshold ({P['MIN_ATR_THRESHOLD']:.3f}).")
        return 0, 0, liquid_equity

    # Calculate Risk-based Size
    risk_dollars = liquid_equity * P['RISK_PER_TRADE']
    risk_distance = atr_value * P['ATR_SL']
    position_size_units_risk = risk_dollars / risk_distance

    # Check Minimum Risk/Reward (R:R) - Only applies if TP and SL are fixed multiples
    tp_distance = atr_value * P['ATR_TP']
    min_rr = P.get('WFO_MIN_RISK_REWARD', 0.0)
    actual_rr = tp_distance / risk_distance
    if min_rr > 0.0 and actual_rr < min_rr:
        print(f"🚫 POS SIZING BLOCKED: {symbol}. R:R ({actual_rr:.2f}) < Min R:R ({min_rr:.2f}) based on ATR multiples.")
        return 0, 0, liquid_equity

    # Calculate Volatility-based Size (WFO_VOLATILITY_FACTOR integration)
    volatility_factor = P.get('WFO_VOLATILITY_FACTOR', 0.0)
    if volatility_factor > 0.0:
        # Volatility size: Factor * Liquid Equity / Current Price
        # Assumes WFO_VOLATILITY_FACTOR is the desired position value as a % of capital
        position_value_vol = liquid_equity * volatility_factor
        position_size_units_vol = position_value_vol / current_price

        # Take the minimum of risk-based and volatility-based size
        position_size_units = min(position_size_units_risk, position_size_units_vol)
        print(f"📏 Risk Size: {position_size_units_risk:.6f}, Vol Size: {position_size_units_vol:.6f}. Using Min Size.")
    else:
        position_size_units = position_size_units_risk

    # Calculate Max Equity-based Size (based on total starting capital)
    max_units_by_equity = (TOTAL_START_CAPITAL * P['MAX_POS_SIZE']) / current_price

    # Final Size is the minimum of all calculated sizes
    final_size = min(position_size_units, max_units_by_equity)

    position_value = final_size * current_price

    # Final check to ensure we don't exceed liquid equity
    if position_value > liquid_equity:
        final_size = liquid_equity / current_price
        position_value = final_size * current_price

    print(f"📏 {symbol} Position Size: {final_size:.6f} units, Value: ${position_value:.2f}")
    return final_size, risk_distance, liquid_equity

def smart_position_sizing(exchange, current_price, atr_value, P, config, side, symbol):
    # Fetch WFO Sizing Method
    sizing_method = P.get('WFO_DYNAMIC_POS_SIZING', 'risk_based')

    base_size, risk_dist, liquid_equity = calculate_position_size(exchange, current_price, atr_value, P, config, side, symbol)

    # Acknowledge WFO Sizing Method but currently apply concentration only
    if sizing_method != 'risk_based':
        print(f"🔧 WFO Sizing Method '{sizing_method}' preferred, applying concentration adjustment.")

    # Apply Concentration Adjustment (part of the existing logic, kept for risk management)
    active_count = len(POSITION_INFO)
    if active_count >= MAX_ASSET_CORRELATION_COUNT:
        concentration_factor = max(0.5, 1.0 - (active_count * 0.1))
        base_size *= concentration_factor
        print(f"🔻 Concentration adjustment: {concentration_factor:.1f}x (Active Assets: {active_count})\n")

    return base_size, risk_dist, liquid_equity

def update_performance_analytics():
    global last_portfolio_value
    total_unrealized_pnl = sum(trade.get('unrealized_pnl', 0.0) for trade in POSITION_INFO.values())
    current_portfolio_value = VIRTUAL_BUY_CAPITAL + VIRTUAL_SELL_CAPITAL + total_unrealized_pnl
    PERFORMANCE_METRICS['max_portfolio_value'] = max(PERFORMANCE_METRICS['max_portfolio_value'], current_portfolio_value)
    current_drawdown = (PERFORMANCE_METRICS['max_portfolio_value'] - current_portfolio_value) / PERFORMANCE_METRICS['max_portfolio_value']
    PERFORMANCE_METRICS['max_drawdown'] = max(PERFORMANCE_METRICS['max_drawdown'], current_drawdown)
    hourly_return = (current_portfolio_value - last_portfolio_value) / last_portfolio_value
    PERFORMANCE_METRICS['hourly_returns'].append(hourly_return)
    last_portfolio_value = current_portfolio_value
    print(f"📊 Portfolio: Value=${current_portfolio_value:.2f}, Drawdown={current_drawdown:.2%}")

def record_trade_performance(symbol, side, entry_price, exit_price, size, pnl, exit_reason, duration_hours):
    global PERFORMANCE_METRICS
    trade_record = {
        'timestamp': get_timestamp(), 'symbol': symbol, 'side': side,
        'entry_price': entry_price, 'exit_price': exit_price, 'size': size,
        'pnl': pnl, 'exit_reason': exit_reason, 'duration_hours': duration_hours
    }
    TRADE_HISTORY.append(trade_record)
    PERFORMANCE_METRICS['total_trades'] += 1
    PERFORMANCE_METRICS['total_pnl'] += pnl
    if pnl > 0: PERFORMANCE_METRICS['winning_trades'] += 1
    PERFORMANCE_METRICS['asset_performance'][symbol]['trades'] += 1
    PERFORMANCE_METRICS['asset_performance'][symbol]['pnl'] += pnl
    print(f"📊 Trade Count: {symbol} ({PERFORMANCE_METRICS['asset_performance'][symbol]['trades']}/~{int(187 if symbol == 'SOL/USD' else 188 if symbol == 'ETH/USD' else 222)} per year)")

def update_virtual_equity(entry_price, exit_price, side, size):
    global VIRTUAL_BUY_CAPITAL, VIRTUAL_SELL_CAPITAL, CYCLE_PNL_BUY, CYCLE_PNL_SELL
    if side == 'buy':
        pnl = (exit_price - entry_price) * size
        VIRTUAL_BUY_CAPITAL += pnl
        CYCLE_PNL_BUY += pnl
    else:
        pnl = (entry_price - exit_price) * size
        VIRTUAL_SELL_CAPITAL += pnl
        CYCLE_PNL_SELL += pnl
    print(f"{get_timestamp()} INFO: PnL: ${pnl:,.2f}")
    return pnl

def execute_trade(exchange, current_price, atr_value, signal, confidence, symbol, P):
    global POSITION_INFO, VIRTUAL_ALLOCATED_BUY, VIRTUAL_ALLOCATED_SELL
    side = signal.lower()
    config = ASSET_PROFILES[symbol]
    if symbol in POSITION_INFO:
        print(f"🚫 EXECUTION BLOCKED: {symbol} already has a position ({POSITION_INFO[symbol]['side'].upper()})")
        return
    size, risk_dist, liquid_equity = smart_position_sizing(exchange, current_price, atr_value, P, config, side, symbol)
    if size == 0:
        print(f"🚫 EXECUTION BLOCKED: Position size calculated as zero for {symbol} {signal}.")
        return
    initial_sl = current_price - risk_dist if side == 'buy' else current_price + risk_dist
    try:
        order_id = f"DRYRUN-{symbol}-{int(time.time())}"
        allocated_amount = size * current_price
        if side == 'buy': VIRTUAL_ALLOCATED_BUY += allocated_amount
        else: VIRTUAL_ALLOCATED_SELL += allocated_amount
        print(f"{get_timestamp()} INFO: DRY RUN: {side.upper()} {symbol}. Allocated: ${allocated_amount:,.2f}")
        POSITION_INFO[symbol] = {
            'id': order_id, 'entry_price': current_price, 'current_sl': initial_sl,
            'side': side, 'start_time': time.time(), 'size': size,
            'allocated_amount': allocated_amount, 'highest_price': current_price,
            'lowest_price': current_price, 'confidence': confidence,
            'unrealized_pnl': 0.0
        }
        send_email_alert("TRADE ENTRY", f"Entered {side.upper()} {symbol} @ {current_price:.2f}")
    except Exception as e:
        send_email_alert(f"TRADE ENTRY ERROR: {symbol}", f"Error: {e}")

def manage_trade(exchange, current_price, atr_value, symbol, P):
    global POSITION_INFO, VIRTUAL_ALLOCATED_BUY, VIRTUAL_ALLOCATED_SELL
    trade = POSITION_INFO[symbol]
    exit_reason = exit_price_at_close = None

    # 1. Calculate Take Profit Price
    take_profit_price = trade['entry_price'] + (atr_value * P['ATR_TP']) if trade['side'] == 'buy' else trade['entry_price'] - (atr_value * P['ATR_TP'])
    print(f"🎯 {symbol} TP: ${take_profit_price:.2f}, SL: ${trade['current_sl']:.2f}")

    # Determine extreme price and profit distance for stop management
    if trade['side'] == 'buy':
        trade['highest_price'] = max(trade['highest_price'], current_price)
        extreme_price, profit_dist = float(trade['highest_price']), float(current_price - trade['entry_price'])
    else:
        trade['lowest_price'] = min(trade['lowest_price'], current_price)
        extreme_price, profit_dist = float(trade['lowest_price']), float(trade['entry_price'] - current_price)

    # 2. Check for Exits (TP/SL/Time Stop)
    if trade['side'] == 'buy':
        if current_price >= take_profit_price:
            exit_reason, exit_price_at_close = "TAKE PROFIT HIT", current_price
            print(f"🎯 {symbol}: TAKE PROFIT triggered at {current_price:.2f} (Target: {take_profit_price:.2f})\n")
    else:
        if current_price <= take_profit_price:
            exit_reason, exit_price_at_close = "TAKE PROFIT HIT", current_price
            print(f"🎯 {symbol}: TAKE PROFIT triggered at {current_price:.2f} (Target: {take_profit_price:.2f})\n")

    if not exit_reason and ((trade['side'] == 'buy' and current_price <= trade['current_sl']) or
                           (trade['side'] == 'sell' and current_price >= trade['current_sl'])):
        # Use SL price as exit if SL is hit
        exit_reason, exit_price_at_close = "STOP LOSS HIT", trade['current_sl']

    if not exit_reason and (time.time() - trade['start_time']) / 3600 >= P['MAX_HOLD_PERIODS']:
        exit_reason, exit_price_at_close = "TIME STOP", current_price

    # 3. Handle Exit
    if exit_reason:
        allocated_amount = trade['allocated_amount']
        if trade['side'] == 'buy':
            VIRTUAL_ALLOCATED_BUY -= allocated_amount
        else:
            VIRTUAL_ALLOCATED_SELL -= allocated_amount
        pnl_realized = update_virtual_equity(trade['entry_price'], exit_price_at_close, trade['side'], trade['size'])
        duration_hours = (time.time() - trade['start_time']) / 3600
        if "TAKE PROFIT" in exit_reason:
            print(f"💰 {symbol}: {exit_reason}!")
        elif "STOP LOSS" in exit_reason:
            print(f"🛑 {symbol}: {exit_reason}")
        else:
            print(f"⏰ {symbol}: {exit_reason}")
        record_trade_performance(symbol, trade['side'], trade['entry_price'], exit_price_at_close,
                               trade['size'], pnl_realized, exit_reason, duration_hours)
        send_email_alert(f"TRADE EXIT: {symbol}", f"Closed: {exit_reason}. PnL: ${pnl_realized:,.2f}")
        del POSITION_INFO[symbol]
        return

    # 4. Stop Loss Adjustments

    # --- A. WFO Profit Lock Stop (New Logic) ---
    profit_lock_atr_mult = P.get('WFO_PROFIT_LOCK_ATR', 0.0)
    if profit_lock_atr_mult > 0.0:
        profit_lock_target = atr_value * profit_lock_atr_mult

        # Check if trade has moved by the profit lock ATR distance
        has_reached_profit_lock = profit_dist >= profit_lock_target

        if has_reached_profit_lock:
            # Move SL to entry price + a small buffer
            new_sl_lock = trade['entry_price'] + (0.01 if trade['side'] == 'buy' else -0.01)

            # Only move if the new SL is better than the current one (i.e., less risk)
            if (trade['side'] == 'buy' and new_sl_lock > trade['current_sl']) or \
               (trade['side'] == 'sell' and new_sl_lock < trade['current_sl']):
                trade['current_sl'] = new_sl_lock
                print(f"🔒 {symbol}: WFO Profit Lock engaged to entry")

    # --- B. Breakeven Stop (Original Logic - kept for robustness, though slightly redundant now) ---
    breakeven_target = atr_value * P['BREAKEVEN_ATR']
    if profit_dist >= breakeven_target and abs(trade['current_sl'] - trade['entry_price']) > 0.01:
        new_sl_be = trade['entry_price'] + (0.01 if trade['side'] == 'buy' else -0.01)
        if (trade['side'] == 'buy' and new_sl_be > trade['current_sl']) or \
           (trade['side'] == 'sell' and new_sl_be < trade['current_sl']):
            trade['current_sl'] = new_sl_be
            print(f"⚖️ {symbol}: Breakeven stop moved to entry")

    # --- C. Trailing Stop (Original Logic) ---
    trail_distance = extreme_price * P['TRAILING_STOP_MULT']
    if trade['side'] == 'buy':
        new_trailing_sl = extreme_price - trail_distance
        if new_trailing_sl > trade['current_sl']:
            trade['current_sl'] = new_trailing_sl
            print(f"📈 {symbol}: Trailing SL moved to {new_trailing_sl:.2f}")
    else:
        new_trailing_sl = extreme_price + trail_distance
        if new_trailing_sl < trade['current_sl']:
            trade['current_sl'] = new_trailing_sl
            print(f"📉 {symbol}: Trailing SL moved to {new_trailing_sl:.2f}")

def check_max_drawdown(side):
    global VIRTUAL_BUY_CAPITAL, VIRTUAL_SELL_CAPITAL, VIRTUAL_BUY_PEAK, VIRTUAL_SELL_PEAK
    global MDD_PROTECT_BUY, MDD_PROTECT_SELL
    current_capital = VIRTUAL_BUY_CAPITAL if side == 'buy' else VIRTUAL_SELL_CAPITAL
    peak = VIRTUAL_BUY_PEAK if side == 'buy' else VIRTUAL_SELL_PEAK
    protect_switch = MDD_PROTECT_BUY if side == 'buy' else MDD_PROTECT_SELL
    if current_capital > peak:
        if side == 'buy': VIRTUAL_BUY_PEAK, MDD_PROTECT_BUY = current_capital, False
        else: VIRTUAL_SELL_PEAK, MDD_PROTECT_SELL = current_capital, False
        peak = current_capital
    drawdown = (peak - current_capital) / peak
    if drawdown >= GLOBAL_MAX_DRAWDOWN_PCT and not protect_switch:
        print(f"{get_timestamp()} CRITICAL: {side.upper()} MAX DRAWDOWN ({drawdown:.2%}) HIT. TRADING SUSPENDED.")
        send_email_alert(f"MDD PROTECTION: {side.upper()}\n", f"Drawdown {drawdown:.2%} > {GLOBAL_MAX_DRAWDOWN_PCT:.2%}")
        if side == 'buy': MDD_PROTECT_BUY = True
        else: MDD_PROTECT_SELL = True
    elif protect_switch and drawdown < GLOBAL_MAX_DRAWDOWN_PCT * 0.5:
        print(f"{get_timestamp()} RECOVERY: {side.upper()} MDD protection disengaged.")
        if side == 'buy': MDD_PROTECT_BUY = False
        else: MDD_PROTECT_SELL = False
    return MDD_PROTECT_BUY if side == 'buy' else MDD_PROTECT_SELL

def handle_asset_error(symbol, error):
    global error_counters, disabled_assets
    error_counters[symbol] = error_counters.get(symbol, 0) + 1
    print(f"⚠️ Error #{error_counters[symbol]} for {symbol}: {error}")
    if error_counters[symbol] >= MAX_CONSECUTIVE_ERRORS:
        disabled_assets.add(symbol)
        send_email_alert(f"Asset Disabled: {symbol}", "Disabled after 5 consecutive errors")
        print(f"🚫 {symbol} disabled due to repeated errors")

def reset_cycle_metrics():
    global PERFORMANCE_METRICS
    for symbol in ASSET_PROFILES.keys():
        PERFORMANCE_METRICS['asset_performance'][symbol] = {'trades': 0, 'pnl': 0.0}
    PERFORMANCE_METRICS['total_trades'] = 0
    PERFORMANCE_METRICS['winning_trades'] = 0
    PERFORMANCE_METRICS['total_pnl'] = 0.0

def print_open_positions():
    if not POSITION_INFO:
        print("📊 No open positions")
        return
    print("\n📊 Open Positions:")
    for symbol, trade in POSITION_INFO.items():
        unrealized_pnl = trade.get('unrealized_pnl', 0.0)
        print(f"   {symbol}: {trade['side'].upper()} | Entry: ${trade['entry_price']:.2f} | Size: {trade['size']:.6f} | Current SL: ${trade['current_sl']:.2f} | Unrealized PnL: ${unrealized_pnl:.2f}")

REQUIRED_TABLES = [
    'candles',
    'technical_indicators',
    'trade_history',
    'cycle_performance',
    'asset_cycle_performance',
    'bot_state'
]

def verify_database_tables(db_path, required_tables):
    if not os.path.exists(db_path):
        print(f"❌ DATABASE FILE NOT FOUND at: {db_path}")
        print("ACTION: Ensure the trading bot has run at least once.")
        return
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        missing_tables = []
        print(f"✅ Database file found at: {db_path}")
        print("---" * 13)
        for table in required_tables:
            cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table}'")
            if cursor.fetchone():
                print(f"🟢 FOUND: Table '{table}' exists.")
            else:
                print(f"🔴 MISSING: Table '{table}' is NOT present.")
                missing_tables.append(table)
        print("---" * 13)
        conn.close()
        if not missing_tables:
            print("🎉 SUCCESS: All required performance tables are present!")
        else:
            print("⚠️ WARNING: One or more tables required by the Performance Analyzer are missing.")
    except sqlite3.Error as e:
        print(f"❌ DATABASE ERROR: {e}")
        print("ACTION: Check file permissions or disk space.")


#### MAIN EXECUTION #########
def run_trading_bot():
    run_trading_bot.cycle_counter = 0
    print("--- Starting Enhanced Trading Bot with SQLite & Multi-LLM Veto ---\n")
    exchange = initialize_exchange()
    start_cycle = load_bot_state()
    cycle_count = start_cycle
    send_email_alert("Bot Startup", f"Initialized. Capital: ${VIRTUAL_BUY_CAPITAL + VIRTUAL_SELL_CAPITAL:,.2f}")
    print('\n')
    verify_database_tables(DB_PATH, REQUIRED_TABLES)
    print('\n')
    while True:
        try:
            cycle_count += 1
            run_trading_bot.cycle_counter = cycle_count
            current_time = dt.datetime.now(TIMEZONE)
            global CYCLE_PNL_BUY, CYCLE_PNL_SELL
            CYCLE_PNL_BUY = CYCLE_PNL_SELL = 0.0
            # reset_cycle_metrics() # Note: Original code resets metrics every cycle which may skew hourly stats display
            NET_PNL_BUY = VIRTUAL_BUY_CAPITAL - VIRTUAL_BUY_START
            NET_PNL_SELL = VIRTUAL_SELL_CAPITAL - VIRTUAL_SELL_START
            buy_protected = check_max_drawdown('buy')
            sell_protected = check_max_drawdown('sell')
            print(f"\n{'='*60}")
            print(f"CYCLE {cycle_count} - {get_timestamp(current_time)}")
            print(f"{'='*60}")
            print(f"Buy Pool: ${VIRTUAL_BUY_CAPITAL:,.2f} | Liquid: ${get_liquid_usd_equity(exchange, 'buy'):,.2f} | Net PnL: ${NET_PNL_BUY:,.2f}")
            print(f"Sell Pool: ${VIRTUAL_SELL_CAPITAL:,.2f} | Liquid: ${get_liquid_usd_equity(exchange, 'sell'):,.2f} | Net PnL: ${NET_PNL_SELL:,.2f}")
            print(f"RBS Status: BUY={MDD_PROTECT_BUY} | SELL={MDD_PROTECT_SELL}")
            for symbol, config in ASSET_PROFILES.items():
                if symbol in disabled_assets: continue
                try:
                    df = fetch_data_with_history(exchange, symbol)
                    if df is None or df.empty: continue

                    if not apply_volatility_filter(df, symbol): continue

                    current_price = df['close'].iloc[-1]
                    atr_value = df['ATR'].iloc[-1]
                    print(f"🔍 {symbol} | Price: {current_price:.2f} | ATR: {atr_value:.2f}")

                    # WFO LOOKBACK IS USED INSIDE predict_signal
                    signal, confidence, probs = predict_signal(df, symbol, config['P'])

                    # --- MULTI-LLM VETO START ---
                    llm_score = get_llm_sentiment_score(symbol, df)
                    print('\n')

                    # Manage existing positions first (to update unrealized PnL and check for exits)
                    if symbol in POSITION_INFO:
                        trade = POSITION_INFO[symbol]
                        if trade['side'] == 'buy':
                            unrealized_pnl = (current_price - trade['entry_price']) * trade['size']
                        else:
                            unrealized_pnl = (trade['entry_price'] - current_price) * trade['size']
                        trade['unrealized_pnl'] = unrealized_pnl
                        manage_trade(exchange, current_price, atr_value, symbol, config['P'])
                        # Note: We still proceed to checking the LLM and RBA logic below,
                        # allowing new opposing trades or stacking if the original "if symbol in POSITION_INFO: return"
                        # block was intentionally removed by the user in the trade execution flow.
                        # The original bot blocked new trades if one existed. We proceed here
                        # to allow the Veto to operate as a filter for all signals.

                    # Check LLM Veto (Using the Ensemble Score)
                    veto_threshold = abs(config['P']['LLM_VETO_THRESHOLD'])
                    veto_triggered = False

                    if signal == 'BUY' and llm_score < -veto_threshold:
                        print(f"🧠 LLM VETO BLOCKED: {symbol} BUY. Ensemble Score ({llm_score:.2f}) < -Threshold ({-veto_threshold:.2f})")
                        signal = 'HOLD'
                        veto_triggered = True
                    elif signal == 'SELL' and llm_score > veto_threshold:
                        print(f"🧠 LLM VETO BLOCKED: {symbol} SELL. Ensemble Score ({llm_score:.2f}) > +Threshold ({veto_threshold:.2f})")
                        signal = 'HOLD'
                        veto_triggered = True

                    if not veto_triggered and signal in ['BUY', 'SELL']:
                        print(f"✅ LLM VETO PASSED: {symbol} {signal}. Ensemble Score ({llm_score:.2f}) within threshold.")
                    # --- MULTI-LLM VETO END ---


                    # Check Risk-Based Suspension
                    if signal == 'BUY' and buy_protected:
                        print(f"🛡️ RBS OVERRIDE: {symbol} BUY blocked due to MDD")
                        signal = 'HOLD'
                    elif signal == 'SELL' and sell_protected:
                        print(f"🛡️ RBS OVERRIDE: {symbol} SELL blocked due to MDD")
                        signal = 'HOLD'

                    # Execute Trade (If signal is still BUY/SELL and no existing position)
                    if signal in ['BUY', 'SELL'] and symbol not in POSITION_INFO:
                        print(f"🎯 EXECUTE: {symbol} {signal} (Conf: {confidence:.3f}, LLM: {llm_score:.2f})")
                        execute_trade(exchange, current_price, atr_value, signal, confidence, symbol, config['P'])
                    elif signal == 'HOLD':
                        print(f"⏸️ {symbol}: HOLD")
                    elif symbol in POSITION_INFO and signal in ['BUY', 'SELL']:
                         print(f"🚫 EXECUTION BLOCKED: {symbol} already has a position ({POSITION_INFO[symbol]['side'].upper()}) - Stacking prevented in EXECUTE function.")

                    error_counters[symbol] = 0
                except Exception as asset_e:
                    handle_asset_error(symbol, asset_e)
                    continue

            update_performance_analytics()
            save_cycle_performance(cycle_count, current_time)
            print(f"\n--- Cycle {cycle_count} PnL Summary ---")
            print(f"BUY Trades: ${CYCLE_PNL_BUY:,.2f}")
            print(f"SELL Trades: ${CYCLE_PNL_SELL:,.2f}")
            print(f"Total: ${CYCLE_PNL_BUY + CYCLE_PNL_SELL:,.2f}")
            print_open_positions()
            sleep_duration = TRADE_SLEEP_SECONDS
            next_check = current_time + dt.timedelta(seconds=sleep_duration)
            print(f"💤 Sleeping for 30 minutes. Next: {next_check.strftime('%Y-%m-%d %H:%M:%S %Z')}")
            time.sleep(sleep_duration)
        except Exception as e:
            print(f"💥 CRITICAL SYSTEM ERROR: {e}")
            send_email_alert("BOT CRASH", f"Error: {e}")
            time.sleep(600)


if __name__ == "__main__":
    run_trading_bot()

✅ All three LLM clients initialized (DeepSeek, Claude, GPT)
✅ Database initialized with performance and state tables
--- Starting Enhanced Trading Bot with SQLite & Multi-LLM Veto ---

✅ Model loaded for BTC/USD
✅ Model loaded for ETH/USD
✅ Model loaded for SOL/USD
2025-10-15T12:34:33.425562-04:00 INFO: DRY RUN MODE. TOTAL VIRTUAL CAPITAL: $1,000.00
✅ State loaded. Resuming from Cycle 21.
  > Capital: BUY $499.72 / SELL $499.93
  > Open Positions: 3
📧 Alert: [DRY RUN] Bot Startup


✅ Database file found at: /content/drive/MyDrive/TradingBotLogs/trading_bot_MLLM.db
---------------------------------------
🟢 FOUND: Table 'candles' exists.
🟢 FOUND: Table 'technical_indicators' exists.
🟢 FOUND: Table 'trade_history' exists.
🟢 FOUND: Table 'cycle_performance' exists.
🟢 FOUND: Table 'asset_cycle_performance' exists.
🟢 FOUND: Table 'bot_state' exists.
---------------------------------------
🎉 SUCCESS: All required performance tables are present!



CYCLE 21 - 2025-10-15T12:34:36.326548-04:00
B